In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1ARtbsJrrNgPb6g6Mt_vaMG-ZUykF9LBr007vKt3uc-wOoGp3yEY-p8KrHHo

Successfully saved authorization token.


In [3]:
#Import required packages
import geemap
import pandas as pd
import numpy as np
import time
import glob

In [10]:
airsheds = glob.glob("gridextents_shponly/*.shp")
airsheds=['bishek/grids_bishkek.shp']

In [11]:
# Chennai airshed box is already uploaded as a feature on GEE.
#chennai_box = ee.FeatureCollection("projects/ee-saikrishnadammalapati/assets/chennai-box")
#aoi = chennai_box.geometry()

def get_aoi(airshed_shp):
    airshed_box = geemap.shp_to_ee(airshed_shp)
    aoi = airshed_box.geometry()
    return airshed_box, aoi

# The following is a feature at all India level.
#admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
#india = admin2.filter(ee.Filter.eq('ADM0_NAME', 'India'))

**maskClouds** is a function to mask the satellite image, if the pixel has a "cloud_fraction" less than 0.5. This function is taken from [Ujaval Gandhi's GEE code.](https://code.earthengine.google.co.in/0f1259deeb86530cee552817a05e2031)

In [12]:
def maskClouds(image):
    mask = image.select('cloud_fraction').lt(0.1)
    return image.updateMask(mask)

# clip_image function clips the satellite image to our given area of interest (Chennai airshed box in our case)
# https://gis.stackexchange.com/questions/302760/gee-imagecollection-map-with-multiple-input-function
def clip_image(roi):
    def call_image(image):
        return image.clip(roi)
    return call_image

The satellite images are acquired from [**COPERNICUS/S5P/OFFL/L3_SO2**](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_SO2) service. SO2 data is provided in various terms (tropospheric_column_density, stratospheric_column_density etc). I'm taking tropospheric_column_density for now.

In [21]:
def download_tifs(year):


    #year=2020
    airshed_box, aoi = get_aoi('bishek/grids_bishkek.shp')
    
    airshed_name = 'bishkek'#airshed_shp.split('_')[2].split('.')[0]
    tic = time.perf_counter()
    
    if year ==2022:
        max_month=8
    else:
        max_month=13
    for month in range(1,max_month):
        print(month)
    
        #Image Collection - l3_SO2 satellite -- SELECTING only two bands (SO2 Column Number density and Cloud_fraction)
        collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_SO2').select(['SO2_column_number_density', 'cloud_fraction'])
        if month <9:
            startDate = str(year)+'-0'+str(month)+'-01'
            endDate = str(year)+'-0'+str(month)+'-15'
        elif month==9:
            startDate = str(year)+'-0'+str(month)+'-01'
            endDate = str(year)+'-'+str(month)+'-15'
        elif month<12:
            startDate = str(year)+'-'+str(month)+'-01'
            endDate = str(year)+'-'+str(month)+'-15'
        else:
            startDate = str(year)+'-'+str(month)+'-01'
            endDate = str(year)+'-'+str(month)+'-15'

        #Filter image collection -- filtered for date range, chennai_box range,
        fortnight=0
        while fortnight<2:
            filtered = collection.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(aoi))
            #Apply the maskClouds and clip_image function to each image in the image collection.
            cloudMasked = filtered.map(maskClouds).select('SO2_column_number_density')
            clipped_images = cloudMasked.map(clip_image(aoi))
        
            #fortnightly mean
            image = clipped_images.mean()
        
            #Export image
            geemap.ee_export_image(image, filename='bishek/SO2_tifs/'+airshed_name+'_15dayavg_'+'so2_'+startDate+'.tif',
                               scale=30,
                               region=aoi, file_per_band=True)
        
            ## To download aggregated data for the given airshed box in the form of a csv. Use 'toBands' of above to use this.
            geemap.zonal_statistics(clipped_images.toBands(), airshed_box,
                            'bishek/SO2_csvs/'+airshed_name+'_15dayavg'+'_so2_'+startDate+'.csv', statistics_type='MEAN', scale=30)
        

            print(startDate+'xx'+endDate)
            startDate = startDate[:-2]+'16'
            if month==2:
                endDate = endDate[:-2]+'28'
            elif month in [4,6,9,11]:
                endDate = endDate[:-2]+'30'
            else:
                endDate = endDate[:-2]+'31'
        
            fortnight = fortnight+1

    
        # To download all tif images of a collection 
        #geemap.ee_export_image_collection(clipped_images, out_dir='tifs',
                                     #scale=30
         #                                )

    toc = time.perf_counter()
    print('Time taken {} seconds'.format(toc-tic))

In [22]:
from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing.pool import ThreadPool

In [23]:
airsheds[19:20]

[]

In [24]:
pool= ThreadPool(processes=30)
pool.map(download_tifs,[2021,2022])
#download_tifs('gridextents_shponly\\grids_chennai.shp',year)
#Parallel(n_jobs=mp.cpu_count())(delayed(download_tifs)(airshed_shp,year) for airshed_shp in airsheds)

1
1
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_csvs\bishkek_15dayavg_so2_2021-01-01.csv
2021-01-01xx2021-01-15
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_csvs\bishkek_15dayavg_so2_2022-01-01.csv
2022-01-01xx2022-01-15
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_csvs\bishkek_15dayavg_so2_2021-01-

Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_csvs\bishkek_15dayavg_so2_2022-04-01.csv
2022-04-01xx2022-04-15
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_csvs\bishkek_15dayavg_so2_2021-04-01.csv
2021-04-01xx2021-04-15
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_csvs\bishkek_15dayavg_so2_2022-04-16.csv
2022-04-16xx2022-04-30
5
Generating URL ...
Data downloaded to D:\P

Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_csvs\bishkek_15dayavg_so2_2021-07-01.csv
2021-07-01xx2021-07-15
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_csvs\bishkek_15dayavg_so2_2022-07-16.csv
2022-07-16xx2022-07-31
Time taken 3047.733796299999 seconds
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_csvs\bishkek_15dayavg_so2_2021-07-16.csv
2021-07-16xx2021-07-31
8
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_tifs
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\bishek\SO2_

[None, None]

Ujaval Gandhi used "image.projection().nominalScale().getInfo()" as scale while exporting images. I'm not getting satisfactory results with that scale (huge pixels). So I used a scale of 30 and it gave me intuitively good results. Need help with tuning this parameter.

In [56]:

## Following functions are useful when we download zonal statistics from GEE. 
def prep_dataset(csv_file,metric_name,separator,date_pos=0):
    df = pd.read_csv(csv_file)
    df = df.T.reset_index()[:-13]
    df = df.drop_duplicates()
    df['date']=df['index'].str.split(separator).str[date_pos]
    if df['date'][0][:1]=='A':
        df['year'] = df['date'].str[1:5]
        df['day'] = df['date'].str[5:]
        # converting to date
        df['date'] = pd.to_datetime(df['year'].astype(int) * 1000 + df['day'].astype(int), format='%Y%j')
        df = df.drop(['year','day'],axis=1)
    elif len(df['date'][4])>7:
        df['date']=df['date'].str[:4]+"-"+df['date'].str[4:6]+"-"+df['date'].str[6:]
    else:
         df['date']=df['date'].str[:4]+"-"+df['date'].str[4:6]+"-"+"01"
    
    df = df.drop(['index'],axis=1)
    
    df['date'] = pd.to_datetime(df['date'])
    df.columns = [metric_name,'date']
    
    df = df.groupby('date')[metric_name].mean().reset_index()
    df = df.set_index('date')
    
    df = df.fillna(np.NaN)
    return df

In [25]:
## Preparing dataset
#global chennai_no2_df
#chennai_no2_df = prep_dataset('Chennai_NO2_2020-01-01_2020-01-31.csv','tropospheric_NO2_column_number_density','_')
#df = chennai_no2_df.resample('SMS').mean()
#df.index += pd.Timedelta(14, 'd')

In [85]:
## FOR BISHKEK 
def prep_dataset(csv_file,metric_name,separator,date_pos=0):
    df = pd.read_csv(csv_file)
    df = df.T.reset_index()[:-11]
    df = df.drop_duplicates()
    dff = pd.DataFrame()
    dff['index'] = df['index']
    dff['mean_value'] = df.mean(axis=0)
    
    df = dff
    df['date']=df['index'].str.split(separator).str[date_pos]
    if df['date'][0][:1]=='A':
        df['year'] = df['date'].str[1:5]
        df['day'] = df['date'].str[5:]
        # converting to date
        df['date'] = pd.to_datetime(df['year'].astype(int) * 1000 + df['day'].astype(int), format='%Y%j')
        df = df.drop(['year','day'],axis=1)
    elif len(df['date'][4])>7:
        df['date']=df['date'].str[:4]+"-"+df['date'].str[4:6]+"-"+df['date'].str[6:]
    else:
         df['date']=df['date'].str[:4]+"-"+df['date'].str[4:6]+"-"+"01"
    
    df = df.drop(['index'],axis=1)
    
    df['date'] = pd.to_datetime(df['date'])
    df.columns = [metric_name,'date']
    
    df = df.groupby('date')[metric_name].mean().reset_index()
    df = df.set_index('date')
    
    df = df.fillna(np.NaN)
    return df

In [124]:
import glob
import pandas as pd
import numpy as np
result = glob.glob("bishek\\O3_csvs\*.csv")

In [125]:
master = pd.DataFrame(columns=['date','tropospheric_O3_column_number_density'])
for csv in result:
    df = prep_dataset(csv,'tropospheric_O3_column_number_density','_')
    df = df.resample('SMS').mean()
    #df.index += pd.Timedelta(14, 'd')
    df = df.reset_index()
    master = master.append(df)


master = master.reset_index(drop=True).sort_values(by='date')
master.columns=['start_date','tropospheric_O3_column_number_density']

C:\Users\saikr\AppData\Local\Temp/ipykernel_10344/1039890849.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dff['mean_value'] = df.mean(axis=0)


In [126]:
master.groupby('start_date')[['tropospheric_O3_column_number_density']].mean().to_csv('bishek/Bishkek_2020-22_O3.csv')

In [21]:
import glob
tifs = glob.glob("SO2_tifs/*.tif")

In [22]:
cities = []
for tif in tifs:
    cities.append(tif.split('15')[0].split('\\')[1])

In [23]:
df = pd.DataFrame(cities,columns=['city'])

In [24]:
df.value_counts()#.to_csv('value_coints.csv')

city         
agra_            62
nagaon_          62
pathankot_       62
paontasahib_     62
ongole_          62
                 ..
guwahati_        62
gulburga_        62
gorakhpur_       62
gaya_            62
vizianagaram_    62
Length: 101, dtype: int64

In [ ]:
#2022
#-- Pathankot only 01-01 tif got made.
#-- mumbai - first both tifs in 01 not made.
#-- Latur -- only 01-01 and 06-16 got made.
#-- Nalbari only 4 tifs frmo 01 and 02 got made.
#--Indore -- one tif 01-01 is not made.
#-- Delhi -- one tif 01-01 is not made.

In [31]:
li.map(square)

AttributeError: 'list' object has no attribute 'map'

In [49]:
airshed_shp_temp = 'gridextents_shponly\\grids_agra.shp'
airshed_box, aoi = get_aoi(airshed_shp_temp)
airshed_name = airshed_shp_temp.split('_')[2].split('.')[0]
collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_O3').select(['O3_column_number_density', 'cloud_fraction'])

startDate = '2021-04-16'
endDate = '2021-04-30'

filtered = collection.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(aoi))
#Apply the maskClouds and clip_image function to each image in the image collection.
cloudMasked = filtered.map(maskClouds).select('O3_column_number_density')
clipped_images = cloudMasked.map(clip_image(aoi))
#fortnightly mean
image = clipped_images.mean()
#Export image
geemap.ee_export_image(image, filename='O3_tifs/'+airshed_name+'_15dayavg_'+'o3_'+startDate+'.tif',
                               scale=30,
                               region=aoi, file_per_band=True)
## To download aggregated data for the given airshed box in the form of a csv. Use 'toBands' of above to use this.
geemap.zonal_statistics(clipped_images.toBands(), airshed_box,
                        'O3_csvs/'+airshed_name+'_15dayavg'+'_o3_'+startDate+'.csv', statistics_type='MEAN', scale=30)

Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\O3_tifs
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\O3_csvs\agra_15dayavg_o3_2021-04-16.csv
